# Tarea uso de pipelines para evitar filtración de datos

Cargue el dataset de vehículos de los años 70 y 80, disponible en el siguiente enlace: https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data-original.

Elimine los datos nulos de la variable 'mpg'.

Configure como índice la variable 'car_name'.

Identifique que variables son numéricas, cuáles nominales y cuáles ordinales.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data-original", sep='\s+', header=None, na_values=['?'],
                 names=['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year', 'origin', 'car_name'])
df = df[df['mpg'].notnull()]
df.set_index('car_name', inplace=True)
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin
car_name,,,,,,,,
chevrolet chevelle malibu,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0
buick skylark 320,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0
plymouth satellite,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0
amc rebel sst,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0
ford torino,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0


Cree un modelo de regresión Ridge para predecir la variable 'mpg', usando como variables predictoras las demás disponibles.

Parta los datos en conjuntos de entrenamiento y prueba en una proporción 80/20, usando random_state=1.

El preprocesamiento de las variables debe incluir los siguientes procesos:
- Debe imputar a los datos nulos de 'horsepower' la media de la variable.
- Debe codificar las variables categóricas, de acuerdo con su tipo.
- Debe generar nuevas características polinómicas de grado 2 de las variables numéricas.
- Debe estandarizar las variables numéricas, incluyendo las nuevas polinómicas generadas.
- Debe hacer selección de características mediante un modelo Lasso, eliminando al menos un par de características.

Todo esto lo debe hacer dentro de un **pipeline** para evitar filtración de datos.

Al final debe reportar:
- El error de validación y el valor de $\lambda$ sintonizado.
- El error de prueba.
- Los coeficientes asociados a cada característica del modelo, incluyendo el intercepto.

In [ ]:
# Extraccion de matrices de caracteristicas y objetivo, y division de los datos en entrenamiento y prueba
from sklearn.model_selection import train_test_split
X = df.drop(['mpg'], axis=1)
y = df['mpg'] # Esta es la variable que se va a predecir

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, train_size=0.8)
print(f'Tamaño del conjunto de entrenamiento es: {X_train.shape}')
print(f'Tamaño del conjunto de prueba es: {X_test.shape}')

Tamaño del conjunto de entrenamiento es: (318, 7)
Tamaño del conjunto de prueba es: (80, 7)


Haciendo únicamente imputación de datos con el método SimpleImputer:

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

num_vars = ['displacement', 'horsepower', 'weight', 'acceleration'] # Se definen las variables numéricas
imputer = SimpleImputer(strategy='mean')
poly = PolynomialFeatures(degree=2, include_bias=False)
ohe = OneHotEncoder(sparse_output=False, drop='first')
oe_cyl = OrdinalEncoder(categories=[[3., 4., 5., 6., 8.]])
oe_my = OrdinalEncoder(categories=[[70., 71., 72., 73., 74., 75., 76., 77., 78., 79., 80., 81., 82.]])
ss = StandardScaler()
pre_num = Pipeline([('imputer', imputer), ('poly', poly), ('ss', ss)]) # Se define un pipeline para las variables numéricas

preprocessor = ColumnTransformer(transformers=[('ohe', ohe, ['origin']), # Codificacióp OneHot para la variable 'origin'
                                               ('oe_cylinders', oe_cyl, ['cylinders']), # Codificación ordinal para la variable 'cylinders'
                                               ('oe_model_year', oe_my, ['model_year']), # Codificación ordinal para la variable 'model_year'
                                               ('pre_num', pre_num, num_vars)], # Prepocesamiento de las variables numéricas
                                               remainder='passthrough') # El resto de las columnas se mantienen sin cambios
pipe = Pipeline([('preprocessor', preprocessor), ('model', Ridge())]) # Se define un pipeline con el preprocesador y el modelo Ridge
alpha = np.logspace(-4, 2)
grid = {'model__alpha':alpha} # Se define la grilla de hiperparámetros a sintonizar
grid_search = GridSearchCV(estimator=pipe, # Se define el modelo a sintonizar, que incluye el preprocesador y el modelo Ridge
                           param_grid=grid, # Se define la grilla de hiperparámetros que se va a sintonizar
                           cv=5, # Se define el número de folds en la validación cruzada
                           scoring='neg_root_mean_squared_error') # Se define la métrica de evaluación, en este caso RMSE
grid_search.fit(X_train, y_train)

print(f'Mejor RMSE obtenido fue {-grid_search.best_score_:.3} con hiperparámetros de {grid_search.best_params_}.')
print(f'Error de prueba: {np.sqrt(mean_squared_error(y_test, grid_search.best_estimator_.predict(X_test))):.3f}')

Mejor RMSE obtenido fue 3.12 con hiperparámetros de {'model__alpha': 1.0985411419875573}.
Error de prueba: 2.636


In [ ]:
pesos = pd.DataFrame(data=grid_search.best_estimator_.named_steps['model'].coef_, # Se extraen los coeficientes del modelo Ridge
                     index=grid_search.best_estimator_.named_steps['preprocessor'].get_feature_names_out().tolist(), # Se extraen los nombres de las variables
                     columns=['Coeficiente'])
pesos.loc['intercepto'] = grid_search.best_estimator_.named_steps['model'].intercept_ # Se extrae el intercepto del modelo Ridge
pesos

,Coeficiente
ohe__origin_2.0,1.387686
ohe__origin_3.0,1.147852
oe_cylinders__cylinders,0.204825
oe_model_year__model_year,0.849681
pre_num__displacement,-2.070257
pre_num__horsepower,-4.143573
pre_num__weight,-5.649803
pre_num__acceleration,0.048803
pre_num__displacement^2,-0.159959
pre_num__displacement horsepower,1.060915


Con selección de características usando el método SelectFromModel y Lasso:

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge,Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

num_vars = ['displacement', 'horsepower', 'weight', 'acceleration'] # Se definen las variables numéricas
imputer = SimpleImputer(strategy='mean')
poly = PolynomialFeatures(degree=2, include_bias=False)
ohe = OneHotEncoder(sparse_output=False, drop='first')
oe_cyl = OrdinalEncoder(categories=[[3., 4., 5., 6., 8.]])
oe_my = OrdinalEncoder(categories=[[70., 71., 72., 73., 74., 75., 76., 77., 78., 79., 80., 81., 82.]])
ss = StandardScaler()
pre_num = Pipeline([('imputer', imputer), ('poly', poly), ('ss', ss)]) # Se define un pipeline para las variables numéricas

preprocessor = ColumnTransformer(transformers=[('ohe', ohe, ['origin']), # Codificacióp OneHot para la variable 'origin'
                                               ('oe_cylinders', oe_cyl, ['cylinders']), # Codificación ordinal para la variable 'cylinders'
                                               ('oe_model_year', oe_my, ['model_year']), # Codificación ordinal para la variable 'model_year'
                                               ('pre_num', pre_num, num_vars)], # Prepocesamiento de las variables numéricas
                                               remainder='passthrough') # El resto de las columnas se mantienen sin cambios

features_selector = SelectFromModel(Lasso(alpha=0.01, max_iter=10000)) # Se define el modelo Lasso para la selección de características
pipe = Pipeline([('preprocessor', preprocessor), ('features_selection', features_selector), ('model', Ridge())]) # Se define un pipeline con el preprocesador y el modelo Ridge
alpha = np.logspace(-4, 2)
grid = {'model__alpha':alpha} # Se define la grilla de hiperparámetros a sintonizar
grid_search = GridSearchCV(estimator=pipe, # Se define el modelo a sintonizar, que incluye el preprocesador y el modelo Ridge
                           param_grid=grid, # Se define la grilla de hiperparámetros que se va a sintonizar
                           cv=5, # Se define el número de folds en la validación cruzada
                           scoring='neg_root_mean_squared_error') # Se define la métrica de evaluación, en este caso RMSE
grid_search.fit(X_train, y_train)

print(f'Mejor RMSE obtenido fue {-grid_search.best_score_:.3} con hiperparámetros de {grid_search.best_params_}.')
print(f'Error de prueba: {np.sqrt(mean_squared_error(y_test, grid_search.best_estimator_.predict(X_test))):.3f}')
print(f'Número de características seleccionadas: {grid_search.best_estimator_.named_steps["features_selection"].get_support().sum()}')

Mejor RMSE obtenido fue 3.1 con hiperparámetros de {'model__alpha': 0.0001}.
Error de prueba: 2.631
Número de características seleccionadas: 14


In [ ]:
grid_search.best_estimator_.named_steps

{'preprocessor': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe',
                                  OneHotEncoder(drop='first',
                                                sparse_output=False),
                                  ['origin']),
                                 ('oe_cylinders',
                                  OrdinalEncoder(categories=[[3.0, 4.0, 5.0, 6.0,
                                                              8.0]]),
                                  ['cylinders']),
                                 ('oe_model_year',
                                  OrdinalEncoder(categories=[[70.0, 71.0, 72.0,
                                                              73.0, 74.0, 75.0,
                                                              76.0, 77.0, 78.0,
                                                              79.0, 80.0, 81.0,
                                                              82.0]]),
                          

In [ ]:
selected_features = grid_search.best_estimator_.named_steps["features_selection"].get_feature_names_out(input_features=grid_search.best_estimator_.named_steps['preprocessor'].get_feature_names_out()).tolist()
pesos = pd.DataFrame(data=grid_search.best_estimator_.named_steps['model'].coef_, # Se extraen los coeficientes del modelo Ridge
                     index=selected_features, # Se extraen los nombres de las variables
                     columns=['Coeficiente'])
pesos.loc['intercepto'] = grid_search.best_estimator_.named_steps['model'].intercept_ # Se extrae el intercepto del modelo Ridge
pesos

,Coeficiente
ohe__origin_2.0,1.359317
ohe__origin_3.0,1.019239
oe_cylinders__cylinders,0.332566
oe_model_year__model_year,0.863215
pre_num__horsepower,-3.929289
pre_num__weight,-11.073191
pre_num__displacement weight,2.211209
pre_num__displacement acceleration,-1.295685
pre_num__horsepower^2,2.857975
pre_num__horsepower weight,-0.371862
